In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mutual_info_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

%matplotlib inline

In [2]:
df = pd.read_csv('AB_NYC_2019.csv')

In [3]:
numerical = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
        'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
categorical = ['neighbourhood_group', 'room_type']
base = numerical + categorical + ['price']

In [4]:
df_base =  df[base]
df_base = df_base.fillna(0)

In [5]:
df_full_train, df_test = train_test_split(df_base, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [6]:
len(df_train), len(df_val), len(df_test)

(29337, 9779, 9779)

In [7]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = pd.DataFrame(df_train.price, columns = ['price'])
y_train_orig = y_train
y_val = pd.DataFrame(df_val.price, columns = ['price'])
y_test = pd.DataFrame(df_test.price.values, columns = ['price'])

del df_train['price']
del df_val['price']
del df_test['price']

In [9]:
df_base.neighbourhood_group.value_counts()

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

In [10]:
df_train.corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [11]:
y_train['above_average'] = (y_train.price >= 152).astype(int)
y_val['above_average'] = (y_val.price >= 152).astype(int)
y_test['above_average'] = (y_test.price >= 152).astype(int)

del y_train['price']
del y_val['price']
del y_test['price']

In [12]:
for f in categorical:
    print(f + ':' + str(round(mutual_info_score(df_train[f], y_train.above_average),2)))

neighbourhood_group:0.05
room_type:0.14


In [13]:
dv = DictVectorizer(sparse=False)

In [14]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [15]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train)

/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [16]:
model.intercept_[0]

-0.002568573695255903

In [17]:
model.coef_[0].round(3)

array([ 0.003,  0.004, -0.221, -0.091, -0.012, -0.385,  0.087,  1.216,
       -0.795, -0.127, -0.003, -0.043,  1.656, -1.202, -0.457])

In [18]:
y_pred = model.predict_proba(X_val)[:, 1]

In [19]:
price_decision = (y_pred >= 0.5)

In [20]:
y_val = np.array(y_val)
accuracy = (y_val == price_decision).mean()
accuracy

0.5785052231261487

In [21]:
all_features = categorical + numerical
min_accuracy = float('inf')
min_feat = ''
max_accuracy = 0
max_feat = ''

for x in all_features:
    allminusx = all_features.copy()
    allminusx.remove(x)
    
    train_dict = df_train[allminusx].to_dict(orient= 'records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[allminusx].to_dict(orient = 'records')
    X_val = dv.fit_transform(val_dict)
    
    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    
    price_decision = (y_pred >= 0.5)
    
    y_val = np.array(y_val)
    new_accuracy = abs((y_val == price_decision).mean() - accuracy)
    
    print(x + ' : ' + str(new_accuracy))
    if (new_accuracy > max_accuracy):
      max_accuracy = new_accuracy
      max_feat = x
    if (new_accuracy < min_accuracy):
      min_accuracy = new_accuracy
      min_feat = x
        
print ("Max Accuracy for " + max_feat + " : " + str(max_accuracy))
print ("Min Accuracy for " + min_feat + " : " + str(min_accuracy))
       


/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


neighbourhood_group : 0.033229724074560374


/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


room_type : 0.0725120050341298


/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

reviews_per_month : 0.0003955919532685481


/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


calculated_host_listings_count : 0.0006725063205565984


/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/vioreld/anaconda3/envs/ml-zoomcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


availability_365 : 0.009929358027041202
Max Accuracy for room_type : 0.0725120050341298
Min Accuracy for number_of_reviews : 0.0002769143672879393


In [23]:
y_train = y_train_orig
y_train = np.log1p(y_train)

In [24]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [30]:
values = [0, 0.01, 0.1, 1, 10]
for x in values:
    model = Ridge(alpha = x)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse_x = round(rmse(y_val, y_pred), 3)
    print(str(x) + ' : ' + str(rmse_x))

0 : 0.399
0.01 : 0.399
0.1 : 0.399
1 : 0.399
10 : 0.4
